In [4]:
import duckdb
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import confusion_matrix, classification_report


In [5]:
# Create the database connection
con = duckdb.connect(".config/nfl.duckdb")

In [8]:
# Verify the database state and tables
con.sql("""
    SELECT * 
    FROM duckdb_tables()
""")

┌───────────────┬──────────────┬─────────────┬────────────┬─────────────────┬───────────┬─────────┬───────────────────────┬──────────┬───────────┬─────────────────┬────────────────┬──────────────┬─────────────┬────────────────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [13]:
# Examine the tables and select one for EDA
con.sql("""
    SELECT MAX(time)
    FROM bronze.tracking_week_1
    LIMIT 10
""")

┌───────────────────────┐
│      max("time")      │
│       timestamp       │
├───────────────────────┤
│ 2022-09-13 03:11:56.3 │
└───────────────────────┘